In [1]:
import gzip
import pandas as pd
import json
import re, os
import pyBigWig
import glob
import genebe as gnb

HEADER_HASH = '#'
hgnc = pyBigWig.open("project/aux/hgnc.bb")
    
def is_in_gene_set(pos,gene_list):
    is_gene = False
    for gene in gene_list:
        chromo = gene.split(':')[0]
        if pos['chromosome'] != chromo: continue
        index = gene.split(':')[1]
        start = index.split('-')[0]
        end = index.split('-')[1]
        if pos['position'] >= int(start) and pos['position'] <= int(end):
            is_gene = True
            break
    return is_gene

def get_gene_name(chrom,pos):
    return hgnc.entries(chrom,pos,pos+1)[0][2].split('\t')[6], hgnc.entries(chrom,pos,pos+1)[0][2].split('\t')[7]
    
def parse_annotated_json(path, gene_subset):
    #Takes in a annotated VCF file and filters only variants that lie within a gene subset
    #Gene_subset is expected to be a list of genome positions in the format of chr:start-end
    header = ''
    positions = []
    genes = []
    is_header_line = True
    is_position_line = False
    is_gene_line = False
    gene_section_line = '],"genes":['
    end_line = ']}'
    with gzip.open(file, 'rt') as f:
        position_count = 0
        gene_count = 0
        count = 0
        for line in f:
            trim_line = line.strip()
            if is_header_line:
                ## only keep the "header" field content from the line
                header = trim_line[10:-14]
                is_header_line = False
                is_position_line = True
            elif trim_line == gene_section_line:
                is_gene_line = True
                is_position_line = False
                continue
            elif trim_line == end_line:
                break
            else:
                if is_position_line:
                    ## remove the trailing ',' if there is
                    trim_line = trim_line.rstrip(',')
                    pos = json.loads(trim_line)
                    if is_in_gene_set(pos,gene_subset):
                        positions.append(trim_line)
                        position_count += 1
                    #if position_count == 100000: break
                if is_gene_line:
                    ## remove the trailing ',' if there is
                    genes.append(trim_line.rstrip(','))
                    gene_count += 1
    return {'header':header, 'positions':positions, 'genes':genes}

def get_clingen_annot(records):
    labels = []
    for rec in records:
        labels.append(rec['clinicalInterpretation'])
    return(max(set(labels), key=labels.count))
def get_clinvar_annot(records):
    labels = []
    for rec in records:
        labels = labels + rec['significance']
    return(max(set(labels), key=labels.count))

In [2]:
files = glob.glob('project/results/germline/*/*/*.hard-filtered.vcf.annotated.json.gz')
gene_set = pd.read_csv("project/aux/Mito-Lyso-Pesticide_PD_genes.csv")

In [ ]:
for file in files:
    sample_name = file.split('/')[4]
    if os.path.exists(f'germline_variants/snv/{sample_name}_germline_snv.csv'):
        print(f"Skipping {sample_name}...")
        continue
    df = []
    annotated_vcf = parse_annotated_json(file, gene_set.pos)
    for pos in annotated_vcf['positions']:
        x = json.loads(pos)
        for var_dict in x['variants']:
            build_dict = {'vid' : var_dict['vid'],
                          'chromosome' : var_dict['chromosome'],
                          'begin' : var_dict['begin'],
                          'end' : var_dict['end'],
                          'gene_symbol' : get_gene_name(var_dict['chromosome'],int(var_dict['begin']))[0],
                          'gene_name' : get_gene_name(var_dict['chromosome'],int(var_dict['begin']))[1],
                          'refAllele' : var_dict['refAllele'],
                          'altAllele' : var_dict['altAllele'],
                          'variantType' : var_dict['variantType'],
                          'samples' : x['samples']}
            if 'mappingQuality' in x:
                build_dict = build_dict | {'mappingQuality' : x['mappingQuality']}
            if 'dbsnp' in var_dict:
                build_dict = build_dict | {'dbsnp' : var_dict['dbsnp']}
            if 'hgvsg' in var_dict:
                build_dict = build_dict | {'hgvsg' : var_dict['hgvsg']}
            if 'revel' in var_dict:
                build_dict = build_dict | {'revel' : var_dict['revel']['score']}
            if 'spliceAI' in var_dict:
                prefixed_dict = {'SpliceAI_'+k: v for k, v in var_dict['spliceAI'][0].items()}
                build_dict = build_dict | prefixed_dict
            if 'phylopScore' in var_dict:
                build_dict = build_dict | {'phylopScore' : var_dict['phylopScore']}
            if 'clinvar' in var_dict: # Alot of information...unsure of how to simplify into tabular. Include all for now
                build_dict = build_dict | {'clinvar' : var_dict['clinvar']}
            if 'gnomad' in var_dict:
                prefixed_dict = {'gnomad_'+k: v for k, v in var_dict['gnomad'].items()}
                build_dict = build_dict | prefixed_dict
            if 'primateAI-3D' in var_dict:
                prefixed_dict = {'primateAI-3D_'+k: v for k, v in var_dict['primateAI-3D'][0].items()}
                build_dict = build_dict | prefixed_dict
            if 'oneKg' in var_dict:
                prefixed_dict = {'oneKg_'+k: v for k, v in var_dict['oneKg'].items()}
                build_dict = build_dict | prefixed_dict
            if 'regulatoryRegions' in var_dict:
                prefixed_dict = {'regulatoryRegions_'+k: v for k, v in var_dict['regulatoryRegions'][0].items()}
                build_dict = build_dict | prefixed_dict
            df.append(build_dict)
    df = pd.DataFrame(df)

    #Add genebe ACMG annotation
    acmg_df = gnb.annotate_variants_list_to_dataframe(df['vid'], flatten_consequences=True, api_key='ak-EauF7ylVW3aSNDV5IA3ABu0tz',username='gordon.qian@sydney.edu.au')
    df['acmg_score'] = acmg_df['acmg_score']
    df['acmg_classification'] = acmg_df['acmg_classification']
    df['acmg_criteria'] = acmg_df['acmg_criteria']
    df['clinvar_disease'] = acmg_df['clinvar_disease']
    df['clinvar_classification'] = acmg_df['clinvar_classification']
    if 'alphamissense_score' in acmg_df.columns:
        df['alphamissense_score'] = acmg_df['alphamissense_score']
    df.to_csv(f'germline_variants/snv/{sample_name}_germline_snv.csv')